## [ Work in progress ... ]

This notebook does not contain up-to-date fns as those in src folder.
<br>
## ML notebook:
- Activation functions
- Linear layers
- Loss functions
- Optimizers

### Other features to be added:
- weight initialization in Linear Layer
- regularization term
- sgd with momentum
- ...

### Activation Functions:
- sigmoid (done)
- tanh 
- relu 
<br/> 
Helpful resource: 
- CS231: http://cs231n.github.io/optimization-2/
- activation fns: https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6
- optimizers: http://ruder.io/optimizing-gradient-descent/index.html#gradientdescentvariants
- regularization: https://chunml.github.io/ChunML.github.io/tutorial/Regularization/

In [487]:
import numpy as np

#### Sigmoid

In [82]:
class Sigmoid(object):
    '''
    '''
    def __init__(self):
        self.grad = None
        
    def forward(self, x):
        f = 1.0/(np.exp(-x) + 1)
        self.grad = (1 - f) * f
        return f
    
    def backward(self, dx):
        ''' dx is downstream gradient, need to compute dx * grad to return to upstream.
            Assume dx and stored grad are of the same dimension.
        '''
        return self.grad * dx
        


In [488]:
## Test:
sig = Sigmoid()
np.random.seed(100)
x = np.random.randn(5,3)
f = sig.forward(x)
grad = sig.grad

LL = LinearLayer(3,3)


h = 1e-5
s2 = Sigmoid()
fx = s2.forward(x)
fxh = s2.forward(x+h)
grad_num = (fxh-fx)/h

#x, f, sig.grad, sig.backward(x*x)
#print(grad_num, '\n'*2, s2.grad)

In [489]:
## Test Linear layer grad:
np.random.seed(100)
x = np.random.randn(5,3)

LL = LinearLayer(3,3)
f = LL.forward(x)
dy = np.ones_like(f)
LL.backward(x)
grad = LL.dw


L2 = LinearLayer(3,3)
h = 1e-5
fx = L2.forward(x)
fxh = L2.forward(x+h)
grad_num = (fxh-fx)/h

#x, f, sig.grad, sig.backward(x*x)
#print(grad_num, '\n'*4, grad)

#### tanh

In [493]:
class Tanh(object):
    '''
    '''
    def __init__(self):
        self.grad = None
        
    def forward(self, x):
        f = np.tanh(x)
        self.grad = 1.0/(np.power(x,2) + 1)
        return f
    
    def backward(self, dx):
        ''' dx is downstream gradient, need to compute dx * grad to return to upstream.
            Assume dx and stored grad are of the same dimension.
        '''
        return self.grad * dx

In [496]:
## Test
tanh = Tanh()
np.random.seed(100)
x = np.random.randn(5,3)
f = sig.forward(x)
grad = sig.grad
x, f, grad

(array([[-1.74976547,  0.3426804 ,  1.1530358 ],
        [-0.25243604,  0.98132079,  0.51421884],
        [ 0.22117967, -1.07004333, -0.18949583],
        [ 0.25500144, -0.45802699,  0.43516349],
        [-0.58359505,  0.81684707,  0.67272081]]),
 array([[0.14807678, 0.58484148, 0.76006498],
        [0.437224  , 0.72737021, 0.62579495],
        [0.55507059, 0.25539484, 0.4527673 ],
        [0.56340714, 0.38745398, 0.60710599],
        [0.35810579, 0.69356665, 0.66211213]]),
 array([[0.12615005, 0.24280192, 0.18236621],
        [0.24605917, 0.19830279, 0.23417563],
        [0.24696723, 0.19016832, 0.24776907],
        [0.24597953, 0.23733339, 0.23852831],
        [0.22986603, 0.21253195, 0.22371966]]))

### Linear Layer:

In [523]:
# Create linear layer

class LinearLayer(object):
    ''' affine transform on input x with parameters w and b: y = wx + b
        input_dim is number of variables (dim of x)
    '''
    def __init__(self, input_dim, output_dim, weight_init = 'randn'):
        ### add util to have all diff weight_init methods
        ### assume below wt initializing method is what we have for now:
        self.w = np.random.randn(input_dim, output_dim) * 0.001
        #print('w: ',self.w, '\n'*2)
        self.b = np.random.randn(1, output_dim) * 0
        self.x = None
        
        self.dw = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        y = np.matmul(x, self.w) + self.b
        #print('y:', y, '\n'*2)
        return y, self.w
    
    def backward(self, dy):
        ''' dy is downstream gradient, need to compute dw, db, and  
            return dx to upstream.
        '''
        self.dw = np.matmul(self.x.T,dy)
        self.db = np.sum(dy, axis = 0, keepdims= True)
        dx = np.matmul(dy, self.w.T)
        return dx
    

In [513]:
## Test:
input_dim = 5
output_dim = 3
x = np.random.randn(10, 5) # 10 examples with 5 variables each
layer = LinearLayer(input_dim, output_dim)

y = layer.forward(x)
# Check shapes:
x = np.ones([10,5])
W = np.ones([5,3])
b = np.ones([1,3])
y = np.matmul(x,W)
dy = y*2

dW = np.matmul(x.T,dy)
dW.shape

db = np.matmul(np.ones([1,10]), dy)
db.shape
np.sum(dy, axis = 0, keepdims= True), db

y, w = layer.forward(x)
#y
#layer.backward(dy)
w.shape, y.shape

w:  [[-8.84269750e-04 -7.48579509e-04  1.52509721e-03]
 [-1.05054924e-03 -1.40497548e-04  2.06360719e-04]
 [ 1.70507522e-03 -1.06588939e-03 -8.31055610e-04]
 [-1.67048481e-04 -2.25160717e-04 -1.26641699e-03]
 [ 1.82334269e-04  4.75717973e-04 -6.24749910e-06]] 


y: [[-4.59946846e-04 -1.15031192e-03 -8.85711789e-04]
 [-1.82562489e-03  1.96608769e-03 -1.09938594e-04]
 [-2.29376272e-03 -1.40680675e-03 -4.61076197e-04]
 [ 1.21778434e-03  9.78746217e-05 -1.58001465e-03]
 [-2.04805460e-03  4.66421641e-04 -1.83587401e-03]
 [ 1.21454669e-04  6.06178741e-05 -2.65019136e-03]
 [-1.20778953e-03 -4.85946196e-04  2.82319999e-03]
 [ 1.83250974e-03  1.31394664e-03  1.58641066e-04]
 [ 5.34710992e-05 -3.96908010e-04  7.34404860e-04]
 [ 1.38146096e-03  1.15701696e-03 -5.03133546e-03]] 


y: [[-0.00021446 -0.00170441 -0.00037226]
 [-0.00021446 -0.00170441 -0.00037226]
 [-0.00021446 -0.00170441 -0.00037226]
 [-0.00021446 -0.00170441 -0.00037226]
 [-0.00021446 -0.00170441 -0.00037226]
 [-0.00021446 -0.00170

((5, 3), (10, 3))

### Loss Function:

#### MSE:


In [504]:
def MSE(y_pred, y):
    ''' Assume y_pred and y have the same dim and can be batched.
    '''
    b_size = y_pred.shape[0]
    diff = y_pred.reshape(b_size,-1) - y.reshape(b_size,-1)
    print('diff:',diff)
    mse = np.power(diff, 2).mean()
    
    dLoss = 2 * diff # dLoss/dy_pred
    
    return mse, dLoss


In [505]:
## Test
y_pred = np.random.rand(10,1)
y = np.random.randn(10,1)
mse, dLoss = MSE(y_pred, y)
mse, dLoss
#np.random.randn(10).shape[0]

diff: [[-0.83198523]
 [ 1.62616477]
 [ 0.6465016 ]
 [ 1.08943233]
 [-0.71152732]
 [ 2.26167546]
 [ 1.30279548]
 [-0.54377604]
 [-0.91437191]
 [-0.28778255]]


(1.3474748647130856, array([[-1.66397045],
        [ 3.25232954],
        [ 1.29300319],
        [ 2.17886465],
        [-1.42305464],
        [ 4.52335093],
        [ 2.60559096],
        [-1.08755208],
        [-1.82874381],
        [-0.5755651 ]]))

#### MSE with L2 Regularization:
#### MSE:
$J = \frac{1}{2m}\sum_{i=1}^m(h_\theta(X^{(i)})-y^{(i)})^2$
#### $J = \frac{1}{2m}\sum_{i=1}^m(h_\theta(X^{(i)})-y^{(i)})^2+\frac{\lambda}{2m}\sum_{j=1}^n\theta_j^2$

##### Weights:
$\frac{\partial}{\partial \theta_j}J(\theta) = \frac{1}{m}\sum_{i=1}^m(h_\theta^{(i)}(x^{(i)})-y^{(i)}).x_j^{(i)}+\frac{\lambda}{m}\theta_j$

##### Bias:
$\frac{\partial}{\partial \theta_0}J(\theta) = \frac{1}{m}\sum_{i=1}^m(h_\theta^{(i)}(x^{(i)})-y^{(i)})$


##### Weights/ parameters updates:
$\begin{bmatrix}\theta_0\\\theta_1\\\vdots\\\theta_n\end{bmatrix}=\begin{bmatrix}\theta_0\\\theta_1(1-\alpha\frac{\lambda}{m})\\\vdots\\\theta_n(1-\alpha\frac{\lambda}{m})\end{bmatrix}-\frac{\alpha}{m}\begin{bmatrix}\sum_{i=1}^m(h_\theta^{(i)}(x^{(i)})-y^{(i)})\\\sum_{i=1}^m(h_\theta^{(i)}(x^{(i)})-y^{(i)}).x_1^{(i)}\\\vdots\\\sum_{i=1}^m(h_\theta^{(i)}(x^{(i)})-y^{(i)}).x_n^{(i)}\end{bmatrix}$

#### MSE with L1 Regularization:
#### MSE:
$J = \frac{1}{2m}\sum_{i=1}^m(h_\theta(X^{(i)})-y^{(i)})^2$
#### $J = \frac{1}{2m}\sum_{i=1}^m(h_\theta(X^{(i)})-y^{(i)})^2+\frac{\lambda}{2m}\sum_{j=1}^n\theta_j^2$

##### Weights:
$J = \frac{1}{2m}\sum_{i=1}^m(h_\theta(X^{(i)})-y^{(i)})^2+\frac{\lambda}{m}\sum_{j=1}^n|\theta_j|$

##### Bias:


##### Weights/ parameters updates:


### Model:
Test model with 1 hidden layer (batch gradient descent):

In [380]:
# Sample Data:
x = np.random.randn(100,5)
# example: y = x1*x2*x3^3 + x4/x5 - x5
y = np.asarray(x[:,0]*x[:,1]*np.power(x[:,2],3) + x[:,3]/x[:,4] - x[:,4])
#y = np.power(x,3).sum(axis=1)

grad_clip = 1.0

# create our model, for now it's just a list, later i will make it a class

model = []

# let's add the modules to our model
model.append(LinearLayer(5,10)) # add input to hidden layer
model.append(Sigmoid()) # add the nonlinear activation
model.append(LinearLayer(10,20)) # add input to hidden layer
model.append(Sigmoid()) # add the nonlinear activation

model.append(LinearLayer(20,1)) # output layer
maxiter = 10000
lr = 1e-2

for i in range(maxiter):
    xi = x
    
    # Forward pass:
    for mod in model:
        xi = mod.forward(xi)

    # Loss:
    mse, dLoss = MSE(xi, y)
    dx = dLoss

    # Backward pass:
    for mod in reversed(model):
        dx = mod.backward(dx)
    
    # Update:
    for mod in model:
        if hasattr(mod, 'w'):
            mod.w -= np.clip(mod.dw, -grad_clip, grad_clip)*lr
        if hasattr(mod, 'b'):
            mod.b -= np.clip(mod.db, -grad_clip, grad_clip)*lr
    
    if i%100 == 0:
        print('iteration {}, loss:{}'.format(i, mse))
        print(model[0].dw)
    
    

iteration 0, loss:77.207207886045
[[ 4.95988839e-06  1.67796937e-05 -2.81463374e-05 -1.77194311e-05
  -2.55770919e-06 -5.17912121e-05  7.65713697e-05  2.85076233e-05
   5.29130258e-05  4.75060585e-05]
 [ 8.17989821e-06  2.76732669e-05 -4.64192242e-05 -2.92230800e-05
  -4.21820220e-06 -8.54145556e-05  1.26282278e-04  4.70150521e-05
   8.72647559e-05  7.83473991e-05]
 [ 1.30513365e-07  4.41433992e-07 -7.40532026e-07 -4.66258398e-07
  -6.72997283e-08 -1.36262546e-06  2.01462410e-06  7.49964599e-07
   1.39225056e-06  1.24983851e-06]
 [ 5.31293835e-06  1.79741516e-05 -3.01499344e-05 -1.89807537e-05
  -2.73977486e-06 -5.54779378e-05  8.20217623e-05  3.05369998e-05
   5.66795568e-05  5.08877946e-05]
 [-8.93730636e-06 -3.02356826e-05  5.07172622e-05  3.19289306e-05
   4.60877929e-06  9.33234118e-05 -1.37974786e-04 -5.13684874e-05
  -9.53447326e-05 -8.56018585e-05]]
iteration 100, loss:73.42431953963941
[[-0.42015498 -0.4198545  -0.41738445 -0.42379611 -0.41921281 -0.41671659
  -0.41663952 -0.4

In [310]:
class MLP(object):
    '''
    '''
    def __init__(self, input_dim, output_dim, hidden_dim, hidden_layers = 1, activation_type = 'sigmoid'):
        
        if activation_type == 'sigmoid':
            # note: do not instantiate Sigmoid, if we do this, then all activation fns in all 
            # layers will refer to the same instance.
            self.activtion = Sigmoid ### assume we only have sigmoid for now
        
        self.modules = [LinearLayer(input_dim, hidden_dim), self.activation()]
        
        for i in range(hidden_layers):
            # for each hidden layer, we also need to add an activation function
            self.modules.append(LinearLayer(hidden_dim, hidden_dim))
            self.modules.append(self.activation())
            
        self.modules.append(LinearLayer(hidden_dim, output_dim)) # last output layer
      
    def predict(self, x):
        # Forward pass:
        for mod in self.modules:
            x = mod.forward(x)
        return x
    
    def backprop(self, dy):
        dy = dy.reshape(dy.shape[0],-1)
        for mod in reversed(self.modules):
            dy = mod.backward(dy)


In [497]:
## Test:# Sample Data:
#np.random.seed(100)
x = np.random.randn(100,5)
# example: y = x1*x2*x3^3 + x4/x5 - x5
y = np.asarray(x[:,0]*x[:,1]*np.power(x[:,2],3) + x[:,3]/x[:,4] - x[:,4])

mlp = MLP(5, 1, 30, 2)
vars(mlp)
mlp.predict(x)
mlp.backprop(y)

vars(mlp)

{'activtion': __main__.Sigmoid,
 'modules': [<__main__.LinearLayer at 0x126f070b8>,
  <__main__.LinearLayer at 0x126f07208>]}

### Optimizer

#### Batch gradient descent

In [442]:
def batch_gd(x_train, y_train, mlp, loss, grad_clip=1.5, lr = 1e-4, epochs = 3000, print_every=100):
    ''' batch gradient descent
    '''
    
    for i in range(epochs):
        y_pred = mlp.predict(x_train)
        mse, dLoss = loss(y_pred, y_train)
        mlp.backprop(dLoss)
        
        # update:
        for mod in mlp.modules:
            if hasattr(mod, 'w'):
                mod.w -= np.clip(mod.dw,-grad_clip, grad_clip)*lr
            if hasattr(mod, 'b'):
                mod.b -= np.clip(mod.db,-grad_clip, grad_clip)*lr            
    
        if i%print_every == 0:
            print('iteration {}, loss:{}'.format(i, mse))
    

In [456]:
## Test:
x = np.random.randn(100,5)
# example: y = x1*x2*x3^3 + x4/x5 - x5
y = np.asarray(x[:,0]*x[:,1]*np.power(x[:,2],3) + x[:,3]/x[:,4] - x[:,4])
mlp = MLP(5, 1, 30, 2)

batch_gd(x, y, mlp, MSE, lr=8e-4, print_every=200, epochs = 8000)

iteration 0, loss:51.78898280477289
iteration 200, loss:49.20962479105827
iteration 400, loss:49.209566385507834
iteration 600, loss:49.20947955670865
iteration 800, loss:49.209340743924315
iteration 1000, loss:49.20911049281316
iteration 1200, loss:49.208724326760056
iteration 1400, loss:49.20807446566414
iteration 1600, loss:49.20698003971877
iteration 1800, loss:49.20513789403204
iteration 2000, loss:49.20205148589701
iteration 2200, loss:49.19695340519038
iteration 2400, loss:49.18841560826087
iteration 2600, loss:49.17437396907066
iteration 2800, loss:49.15323313205856
iteration 3000, loss:49.12055559307442
iteration 3200, loss:49.0678009362395
iteration 3400, loss:48.985813831173964
iteration 3600, loss:48.85272811720478
iteration 3800, loss:48.61968067410482
iteration 4000, loss:48.15398066198118
iteration 4200, loss:47.06002652201844
iteration 4400, loss:45.109658673769815
iteration 4600, loss:43.94045226609846
iteration 4800, loss:42.95504094370998
iteration 5000, loss:39.3771

#### Stochastic gradient descent

In [524]:
def sgd(x_train, y_train, mlp, loss, grad_clip=1.5, lr = 1e-4, epochs = 3000, print_every=100, shuffle=True):
    ''' sgd
    '''
        
    for i in range(epochs):
        x, y = data_batcher(x_train, y_train, 1, shuffle) # x, y are lists of arrays
        #lr = lr/len(x_train)
        for k, xk in enumerate(x):
            yk_pred = mlp.predict(xk)
            mse, dLoss = loss(yk_pred, y[k])
            mlp.backprop(dLoss)
            
            # update:
            for mod in mlp.modules:
                if hasattr(mod, 'w'):
                    mod.w -= np.clip(mod.dw,-grad_clip, grad_clip)*lr
                if hasattr(mod, 'b'):
                    mod.b -= np.clip(mod.db,-grad_clip, grad_clip)*lr
                
        
        if i%print_every == 0:
            print('iteration {}, loss:{}'.format(i, mse))

In [526]:
## Test:
x = np.random.randn(1000,5)
# example: y = x1*x2*x3^3 + x4/x5 - x5
y = np.asarray(x[:,0]*x[:,1]*np.power(x[:,2],3) + x[:,3]/x[:,4] - x[:,4])
mlp = MLP(5, 1, 30,0)

#sgd(x, y, mlp, MSE, shuffle=True, lr=8e-3, print_every=200)

#### Mini-batch stochastic gradient descent

In [417]:
def mini_batch_sgd(x_train, y_train, batch_size, mlp, loss, grad_clip=1.5, lr = 1e-4, epochs = 3000, print_every=100, shuffle=True):
    ''' mini-batch sgd
    '''
        
    for i in range(epochs):
        x, y = data_batcher(x_train, y_train, batch_size, shuffle) # x, y are lists of arrays
        #lr = lr/len(x_train)
        for k, xk in enumerate(x):
            yk_pred = mlp.predict(xk)
            mse, dLoss = loss(yk_pred, y[k])
            mlp.backprop(dLoss)
            
            # update:
            for mod in mlp.modules:
                if hasattr(mod, 'w'):
                    mod.w -= np.clip(mod.dw,-grad_clip, grad_clip)*lr
                if hasattr(mod, 'b'):
                    mod.b -= np.clip(mod.db,-grad_clip, grad_clip)*lr
                
        
        if i%print_every == 0:
            print('iteration {}, loss:{}'.format(i, mse))
        

In [425]:
## Test:
x = np.random.randn(1000,5)
# example: y = x1*x2*x3^3 + x4/x5 - x5
y = np.asarray(x[:,0]*x[:,1]*np.power(x[:,2],3) + x[:,3]/x[:,4] - x[:,4])
mlp = MLP(5, 1, 30, 0)

mini_batch_sgd(x, y, 64, mlp, MSE, shuffle=True, lr=5e-3, print_every=200)

iteration 0, loss:83.415846181896
iteration 200, loss:10.653716731247536
iteration 400, loss:9.238735472735973
iteration 600, loss:119.24641863065753
iteration 800, loss:75.84298393067365
iteration 1000, loss:17.82380198139399
iteration 1200, loss:38.83090772425603
iteration 1400, loss:86.58561286491633
iteration 1600, loss:5.327346342231616
iteration 1800, loss:13.468480915970392
iteration 2000, loss:5.846953962427725
iteration 2200, loss:5.092312552541605
iteration 2400, loss:21.722123189007835
iteration 2600, loss:5.757869348275998
iteration 2800, loss:831.8037055046827


In [330]:
import torch
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)

AttributeError: module 'torch.utils' has no attribute 'data'

In [337]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])))

Processing...
Done!


In [340]:
a = list(train_loader.batch_sampler)

In [355]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

In [354]:
train_loader

for i, (x, y) in enumerate(train_loader):

tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.424


KeyboardInterrupt



array([[9, 9, 9]])

## Utilities

## Data Batcher
This data batcher class will take in a x and y data set and can generate training data
on demand

In [247]:
def data_batcher(x, y, batch_size, shuffle=False):
    data_len = x.shape[0]
    # reshape checks
    x = x.reshape(data_len,-1)
    y = y.reshape(data_len,-1)
    # check if x and y has same batch shape
    assert x.shape[0] == y.shape[0], 'x and y shapes are not equal'
    # get our shuffled index
    if shuffle:
        idx = np.arange(data_len)
        np.random.shuffle(idx)
        x = x[idx]
        y = y[idx]
    # get number of batches
    num_batches, remainder = divmod(data_len, batch_size)
    x_list = np.array_split(x[:data_len-remainder,:], num_batches)
    y_list = np.array_split(y[:data_len-remainder,:], num_batches)
    if remainder != 0:
        x_list.append(x[data_len-remainder:])
        y_list.append(y[data_len-remainder:])
    return x_list, y_list
    

In [356]:
## Test:
x = np.repeat(np.arange(10).reshape(-1,1),3,axis=1)
y = np.repeat(np.arange(10,20).reshape(-1,1),3,axis=1)
x_list, y_list = data_batcher(x,y,3)

"""
for item in zip(x_list,y_list):
    print('x:{}\n\n y:{}\n\n\n'.format(item[0],item[1]))
"""

print(type(x_list))
for x in x_list:
    print(type(x))
    print(x)

<class 'list'>
<class 'numpy.ndarray'>
[[0 0 0]
 [1 1 1]
 [2 2 2]]
<class 'numpy.ndarray'>
[[3 3 3]
 [4 4 4]
 [5 5 5]]
<class 'numpy.ndarray'>
[[6 6 6]
 [7 7 7]
 [8 8 8]]
<class 'numpy.ndarray'>
[[9 9 9]]


In [518]:
## Test:
x = np.random.randn(10,3)
print(x, '\n'*3)
for k, xk in enumerate(x):
    print(xk, k)

[[ 0.80149953 -0.89929913 -0.18178378]
 [ 0.45625792 -2.29824498 -0.75719966]
 [-0.51234946 -1.52911761  1.57871002]
 [-0.29179517 -2.89151148  1.58134343]
 [ 1.67907091  1.84299483 -0.24104904]
 [-0.47197705 -0.04472072  1.24851755]
 [ 0.2107764  -0.72360079  0.11570072]
 [ 1.56849524  1.99520491 -2.04738692]
 [ 1.04225089 -0.0489416  -0.98832416]
 [ 0.06264861  1.17263108 -2.14590649]] 



[ 0.80149953 -0.89929913 -0.18178378] 0
[ 0.45625792 -2.29824498 -0.75719966] 1
[-0.51234946 -1.52911761  1.57871002] 2
[-0.29179517 -2.89151148  1.58134343] 3
[ 1.67907091  1.84299483 -0.24104904] 4
[-0.47197705 -0.04472072  1.24851755] 5
[ 0.2107764  -0.72360079  0.11570072] 6
[ 1.56849524  1.99520491 -2.04738692] 7
[ 1.04225089 -0.0489416  -0.98832416] 8
[ 0.06264861  1.17263108 -2.14590649] 9
